## Method based on moving average
Data is grouped based on 'Commodity', 'State' and 'Market Center'. The moving average of 'Modal Prices' for 7 day period is calculated. The difference of a particular day's Price  from Moving average is calculated. This difference is compared with the deviation from mean, standard deviation of difference for the group

In [9]:
import pandas as pd
import numpy as np
import statistics
df = pd.read_csv("C://Users//90821//Desktop//Test1lac.csv")
df_concat = df[['Unnamed: 0', 'Commodity','Market Center', 'Modal Prices(in Rs/quintals)', 'State']]
len(df_concat)

100841

In [16]:
flag1 = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['flag1'])
flag2 = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['flag2'])
flag3 = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['flag3'])
MA = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['MA'])
diff = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['difference'])
meandf = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['mean'])
stddf = pd.DataFrame(np.nan, index=np.arange(0,100842,1), columns=['std'])

In [12]:
# Three 'for' loops that iterate a given Commodity and for a State and for a Market Center
list_commodity = df_concat["Commodity"].unique().tolist() 
for q, qval in enumerate(list_commodity):
    d = df_concat[df_concat["Commodity"] == qval ] 
    
    list_state = d["State"].unique().tolist()
    for s, sval in enumerate(list_state):
        b = d[d["State"] == sval]
        
        list_marketcenter = b["Market Center"].unique().tolist()
        for i, val in enumerate(list_marketcenter):

            c = b[b["Market Center"] == val ]

            try:
            
                moving_average = c["Modal Prices(in Rs/quintals)"].rolling(window=7, min_periods = 6, center = True).mean()
                difference = c["Modal Prices(in Rs/quintals)"] - moving_average
                difference = abs(difference)
                std = difference.std()
                mean = difference.mean()
                
                for j in c["Unnamed: 0"]:
                    
                    MA.loc[j]["MA"] = moving_average[j]
                    diff.loc[j]["difference"] = difference[j]
                    meandf.loc[j]["mean"] = mean
                    stddf.loc[j]["std"] = std
                   
                    #defining range upto 1 standard deviation      
                    range_min = mean - std
                    range_max = mean + std
                  
                    if ((diff.loc[j]['difference'] < range_max) & (diff.loc[j]['difference'] > range_min)) :
                        flag1.loc[j]['flag1'] = 0
                    else :
                         flag1.loc[j]['flag1'] = 1
                    
                    #defining range upto 2 standard deviation      
                    range_min = mean - (2*std)
                    range_max = mean + (2*std)
                  
                    if ((diff.loc[j]['difference'] < range_max) & (diff.loc[j]['difference'] > range_min)) :
                        flag2.loc[j]['flag2'] = 0
                    else :
                         flag2.loc[j]['flag2'] = 1
                    
                    #defining range upto 3 standard deviation      
                    range_min = mean - (3*std)
                    range_max = mean + (3*std)
                  
                    if ((diff.loc[j]['difference'] < range_max) & (diff.loc[j]['difference'] > range_min)) :
                        flag3.loc[j]['flag3'] = 0
                    else :
                         flag3.loc[j]['flag3'] = 1
                    
            except:
                #catching exception to print groups with single data points
                print(c["Unnamed: 0"])
                continue


del difference
del std
del mean
del moving_average
del range_min
del range_max
del c
del d

In [13]:
dfcombined = pd.concat([df_concat, diff], axis = 1)
dfcombined = pd.concat([dfcombined, MA], axis = 1)
dfcombined = pd.concat([dfcombined, meandf], axis = 1)
dfcombined = pd.concat([dfcombined, stddf], axis = 1)
dfcombined = pd.concat([dfcombined, flag1], axis = 1)
dfcombined = pd.concat([dfcombined, flag2], axis = 1)
dfcombined = pd.concat([dfcombined, flag3], axis = 1)
dfcombined.dropna(inplace = True)
dfcombined.head(5)

,Unnamed: 0,Commodity,Market Center,Modal Prices(in Rs/quintals),State,difference,MA,mean,std,flag1,flag2,flag3
2,2.0,Coffee,Kalpetta,11000.0,Kerala,-33.333333,11033.333333,0.385356,329.459148,0.0,0.0,0.0
3,3.0,Coffee,Kalpetta,11100.0,Kerala,28.571429,11071.428571,0.385356,329.459148,0.0,0.0,0.0
4,4.0,Coffee,Kalpetta,11100.0,Kerala,0.000000,11100.000000,0.385356,329.459148,0.0,0.0,0.0
5,5.0,Coffee,Kalpetta,11100.0,Kerala,-14.285714,11114.285714,0.385356,329.459148,0.0,0.0,0.0
6,6.0,Coffee,Kalpetta,11300.0,Kerala,114.285714,11185.714286,0.385356,329.459148,0.0,0.0,0.0
7,7.0,Coffee,Kalpetta,11100.0,Kerala,-128.571429,11228.571429,0.385356,329.459148,0.0,0.0,0.0
8,8.0,Coffee,Kalpetta,11100.0,Kerala,-185.714286,11285.714286,0.385356,329.459148,0.0,0.0,0.0
9,9.0,Coffee,Kalpetta,11500.0,Kerala,157.142857,11342.857143,0.385356,329.459148,0.0,0.0,0.0
10,10.0,Coffee,Kalpetta,11400.0,Kerala,28.571429,11371.428571,0.385356,329.459148,0.0,0.0,0.0
11,11.0,Coffee,Kalpetta,11500.0,Kerala,71.428571,11428.571429,0.385356,329.459148,0.0,0.0,0.0


In [14]:
#exporting results in excel 
dfcombined.to_excel("C://Users//90821//Desktop//dfcombined.xlsx")